In [42]:
from lxml import html
import requests
from time import sleep
import json

In [9]:
base_url = "https://cran.r-project.org/"

In [40]:
def get_package_names(base_url, max_iter=10000):
    url = base_url + "web/packages/available_packages_by_name.html"
    page = requests.get(url)
    tree = html.fromstring(page.content)
    
    result = []
    for i in range(2,max_iter):   
        pkg_xpath = '//tr[{0}]/td[1]/a/text()'
        pkg_name = tree.xpath(pkg_xpath.format(i))
        if pkg_name != []:
            result.append(str(pkg_name[0]))
            miss_count = 0
        else: # might be end, but might be just a skipped row
            miss_count += 1
            if miss_count == 2: # if two misses then call it quits
                break
    return result

def build_urls(base_url, package_names):
    
    if type(package_names) == list:
        urls = [base_url + 
                'web/packages/{0}/index.html'.format(x) 
                for x in package_names]
        return urls
    elif type(package_names) == str:
        urls = base_url + 'web/packages/{0}/index.html'.format(package_names) 
        return urls
    else:
        raise TypeError("package_names must be string or list")

def get_depends(base_url, package_names):
    
    
    i = 1
    result = {}
    
    for pkg in package_names:    
        url = build_urls(base_url, pkg)
        page = requests.get(url)
        tree = html.fromstring(page.content)
        depends = tree.xpath("*//table[1]/tr[2]/td[2]/a/text()")
        result[pkg] = depends
        sleep(2)
        i += 1
        if i%100 == 0:
            print("retrieved {0} of {1} package dependencies"\
                  .format(i, len(package_names)))
    
    return(result)
    

In [33]:
a = get_package_names(base_url)
a[:10]

['A3',
 'abbyyR',
 'abc',
 'ABCanalysis',
 'abc.data',
 'abcdeFBA',
 'ABCoptim',
 'ABCp2',
 'abcrf',
 'abctools']

In [34]:
print(a[0], type(a[0]))

A3 <class 'str'>


In [35]:
b = build_urls(base_url, a[0])
b

'https://cran.r-project.org/web/packages/A3/index.html'

In [36]:
c = build_urls(base_url, a)
c[:10]

['https://cran.r-project.org/web/packages/A3/index.html',
 'https://cran.r-project.org/web/packages/abbyyR/index.html',
 'https://cran.r-project.org/web/packages/abc/index.html',
 'https://cran.r-project.org/web/packages/ABCanalysis/index.html',
 'https://cran.r-project.org/web/packages/abc.data/index.html',
 'https://cran.r-project.org/web/packages/abcdeFBA/index.html',
 'https://cran.r-project.org/web/packages/ABCoptim/index.html',
 'https://cran.r-project.org/web/packages/ABCp2/index.html',
 'https://cran.r-project.org/web/packages/abcrf/index.html',
 'https://cran.r-project.org/web/packages/abctools/index.html']

In [41]:
d = get_depends(base_url, a[:10])

In [43]:
with open('test.json', 'w') as fp:
    json.dump(d, fp)

In [44]:
len(d)

10

References:
* http://blog.revolutionanalytics.com/2015/06/fishing-for-packages-in-cran.html
* https://renkun.me/blog/2014/07/29/scraping-information-of-cran-packages.html

In [46]:
with open('test.json','r') as f:
    e = json.load(f)